In [ ]:
import csv
import time
import dask
import dask.dataframe as da

partitionSize = 100000
i = 1
max_fileSize = 12

while(i <= max_fileSize):
    
    try:
        df = da.read_csv(f'/apps/python-docker/dmpfiles/output-{i}.csv',blocksize = None)  #dmp from primary cluster
        print(len(df))                     
        ddf=df.set_index('order_id',compute=True)
        ddf = df.repartition(npartitions=(len(df)//partitionSize))
        ddf.map_partitions(len).compute()  #size (count no of rows in each df)
        
        def name(num):
            num = num+1
            return str(num)

        Batch_Files = ddf.to_csv(r'/apps/python-docker/partition/backup_data_*.csv', name_function=name)
        print(i)
        i+=1
        
    except Exception as e:
        i+=1
        print("Error")         